In [7]:
!pip install matplotlib
!pip install ..
#

ERROR: Directory '..' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [8]:
import numpy as onp

import jax
import jax.numpy as jnp

from jax import random, vmap

import matplotlib

from jax.scipy.stats import norm
from jax.scipy.stats import uniform

from functools import partial

from jax_sgmc import distributions
from jax_sgmc import potential

from jax_sgmc import data

Linear Regression
=========================

Problem
---------

$ y_i = \sum_{j=1}^N w_jx_i + \mathcal{N}(0, \sigma^2),\ i = 1, \ldots, M$

A multiple of samples are used to
deal with overfitting.


Reference Data
---------------

1. Draw weights $w_j$
2. For each sample $i$:
    1. Draw $N$ values of $x_j$
    2. Draw noise $\sigma$
    3. Calculate $y_j$

In [9]:
# Todo: This is not the correct form to handle batches? -> It is possible but it
# is kind of multiple batching

N = 200
M = 100

samples = 1000

sigma = 5.0

key = random.PRNGKey(0)
split, key = random.split(key)

w = random.uniform(split, minval=-1, maxval=1, shape=(N, 1))

def draw_samples(key, w):
    # For each sample draw a new set of predictor variables x and calculate
    # the response

    split1, split2, key = random.split(key, 3)

    noise = sigma * random.normal(split2, shape=(samples, 1))
    x = random.uniform(split1, minval=-10, maxval=10, shape=(samples, N))
    y = jnp.matmul(x,w) + noise

    return x, y

X, Y = draw_samples(key, w)

The reference data and the desired batch shape must be load into a data loader

In [10]:
data_loader = data.NumpyDataLoader(M, x=X, y=Y)

Solution
---------

### Exact

In [11]:
w_sol, res, _, _ = jnp.linalg.lstsq(X, Y)

print(f"Max error: {jnp.max(jnp.abs(w - w_sol))}")

Max error: 0.08426797389984131


### SGLD


In [12]:
# First we need to define the (deterministic) model

def model(sample, parameters):
    weights = sample["w"]

    print(weights)
    print(parameters)

    predictions = jnp.dot(parameters, weights)
    return {"y": predictions}

# The combination of prior and distribution form the posterior, from which we
# sampling is performed.

def likelihood(model_results, extra_parameters, reference_data):
    sigma = extra_parameters["sigma"]
    y = model_results["y"]
    return norm.logpdf(y, loc=reference_data, scale=sigma)

def prior(sample):
    return uniform.logpdf(sample,
                          loc=-10 * jnp.ones(sample.shape),
                          scale=-10 * jnp.ones(sample.shape))

test_w = random.normal(key, (N, 1))

# Get a reference batch

batch_y, batch_x = data_loader.get_random_batch()

print(model({"w": test_w}, batch_x[0,::]))

# We need to define how the posterior can be evaluated by a mini-batch of data
# instead of jsut for a single sample.

# minibatch_potential = potential.minibatch_potential_function(
#     prior,
#     likelihood,
#     strategy="map"
# )

# We test the potential evaluation for a single


AttributeError: 'NumpyDataLoader' object has no attribute 'get_random_batch'